<a href="https://colab.research.google.com/github/cauarichard/crud2/blob/main/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import shutil
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.cluster import KMeans
from google.colab import drive

# 1. Montar o Drive
drive.mount('/content/drive')

# Caminho da pasta onde estão todas as imagens misturadas
base_dir = '/content/drive/MyDrive/dataset/real_fake/faces_geral_real_fake'

# 2. Configurações
# Quantos grupos você quer que ele tente separar?
# Ex: Se colocar 5, ele vai tentar achar 5 padrões diferentes de manipulação/realidade
NUM_GRUPOS = 5

# Carregar o modelo VGG16 pré-treinado (apenas para extrair características, não para classificar)
# include_top=False significa que tiramos a parte final que diz "gato/cachorro" e ficamos só com a "visão"
model = VGG16(weights='imagenet', include_top=False)

print("--- Iniciando processamento das imagens ---")

# Lista para guardar os arquivos e seus vetores de características
file_list = []
feature_list = []

# Varrer a pasta
for file_name in os.listdir(base_dir):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(base_dir, file_name)

        try:
            # Carregar e processar a imagem para o formato que a VGG entende
            img = image.load_img(img_path, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)

            # A mágica acontece aqui: Extrair as características visuais
            features = model.predict(img_data, verbose=0)

            # Transformar o resultado 3D em um vetor plano 1D para o agrupamento
            features_flatten = features.flatten()

            file_list.append(file_name)
            feature_list.append(features_flatten)

        except Exception as e:
            print(f"Erro ao ler {file_name}: {e}")

print(f"\nCaracterísticas extraídas de {len(file_list)} imagens.")
print("--- Iniciando Agrupamento (K-Means) ---")

# 3. Aplicar o algoritmo de agrupamento (K-Means)
kmeans = KMeans(n_clusters=NUM_GRUPOS, random_state=42)
kmeans.fit(feature_list)

# Os 'labels_' aqui não são R, F, etc. São números: 0, 1, 2, 3... (os grupos que ele criou)
grupos = kmeans.labels_

# 4. Mover arquivos para pastas separadas baseadas no que a IA decidiu
print("--- Organizando arquivos nas pastas ---")

caminho_resultado = os.path.join(base_dir, 'resultado_agrupamento')
if not os.path.exists(caminho_resultado):
    os.makedirs(caminho_resultado)

for i, arquivo in enumerate(file_list):
    grupo_id = grupos[i]

    # Criar pasta do grupo (ex: grupo_0, grupo_1)
    pasta_destino = os.path.join(caminho_resultado, f'grupo_{grupo_id}')
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    # Mover (ou copiar) o arquivo
    src = os.path.join(base_dir, arquivo)
    dst = os.path.join(pasta_destino, arquivo)

    shutil.copy(src, dst) # Estou usando COPY para não perder os originais, se quiser mover use shutil.move

print(f"\nConcluído! Vá até a pasta:\n{caminho_resultado}\nE veja como ele separou as imagens.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
--- Iniciando processamento das imagens ---

Características extraídas de 89 imagens.
--- Iniciando Agrupamento (K-Means) ---
--- Organizando arquivos nas pastas ---

Concluído! Vá até a pasta:
/content/drive/MyDrive/dataset/real_fake/faces_geral_real_fake/resultado_agrupamento
E veja como ele separou as imagens.
